In [ ]:
"""
Extract attributes from Google Earth Engine (GEE)

maxwell.cook@colorado.edu
"""

import ee
import geemap

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

print("Success")

# Calculate Sentinel-based MNDWI, EVI, and LAI for fire boundaries

Bring in the FIRED perimeters with at least 5% aspen cover and sufficient FRP observations. Load the Sentinel-2 MSI collection and calculate prefire LAI and MNDWI.

In [ ]:
# Load the FRP observations
frp = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_plots_fired_events_west_aspen')
print(frp)
# Grab a list of fire IDs
fire_ids = frp.aggregate_array('fired_id').distinct()
print(fire_ids.slice(0,10).getInfo())

In [ ]:
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/fired_events_west_aspen')
print(fires)
print(ee.Number(fires.first().get('ig_year').getInfo()).subtract(1).getInfo())

In [ ]:
# Function to calculate the indices
def calc_indices(image):

    # Modified Normalized Difference Water Index (MNDWI)
    # https://www.mdpi.com/2072-4292/8/4/354
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')

    # Enhanced Vegetation Index (EVI)
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))", {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B3')
        }).rename('EVI')

    image = image.addBands([mndwi, evi])

    # Leaf Area Index (LAI)
    lai = image.expression(
        '(3.618 * EVI - 0.118)', {
            'EVI': image.select('EVI')
        }).rename('LAI')

    return image.addBands(lai)

# Function to concatenate strings (rename bands)
def string_cat(item, y):
    return ee.String(item).cat(y)

def apply_string_cat(x, y):
    return x.map(lambda item: string_cat(item, y))

# Function to join collections
def join_collections(col1, col2):
    joined = ee.ImageCollection(ee.Join.saveFirst('cs').apply(
        primary=col1,
        secondary=col2,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    ))
    return joined.map(lambda image: image.addBands(image.get('cs')))


In [ ]:
# Load the S2-MSI Level 1C
s2l1c = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
# Load the Cloud Score Plus
csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

# Sentinel-2 MSI Bands and Vegetation Indices
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

# Define some constants
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.60

# Function to create an image composite in the fire bounds
def generate_composites(fire_id):
    
    # Get the fire perimeter and bounding geometry
    fire = fires.filter(ee.Filter.eq('fired_id', fire_id)).first()
    bounds = fire.geometry().buffer(1000).bounds()

    # Get the VIIRS FRP observations
    frp_fire = frp.filter(ee.Filter.eq('fired_id', fire_id))
    
    # Grab the ignition date information
    ig_year = ee.Number(fire.get('ig_year')).subtract(1)
    
    # Filter the S2-MSI collection
    s2_filter = ee.Filter.And(
      ee.Filter.bounds(bounds),
      ee.Filter.calendarRange(ig_year, ig_year, 'year'),
      ee.Filter.calendarRange(6, 8, 'month'),
      ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80)
    )
    
    # Image Collection
    s2 = s2l1c.filter(s2_filter).select(s2_bands)
    
    # Grab some statistics from the collection
    n = s2.size();
    start_date = ee.Date(s2.first().get('system:time_start'));
    end_date = ee.Date(
      s2.limit(1,'system:time_start',False)
        .first().get('system:time_start'));
    
    # Join the Cloud Score Plus
    csPlus_s = csPlus.filter(
    ee.Filter.And(
      ee.Filter.bounds(bounds),
      ee.Filter.calendarRange(ig_year, ig_year, 'year'),
      ee.Filter.calendarRange(5, 9, 'month'),
    )
    ).select([QA_BAND])
    
    # Mask out occluded pixels
    s2 = join_collections(s2,csPlus_s)
    s2 = s2.map(
      lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD))
    ).select(s2_bands)
    
    # Calculate the spectral indices
    s2 = s2.map(calc_indices).median().clip(bounds).set('fired_id',fire_id)

    # Extract the reduction for spectral indices
    s2_reduc = s2.select(['MNDWI','EVI','LAI']).reduceRegions(
        collection = frp_fire,
        reducer = ee.Reducer.mean(),
        scale = 10,
        tileScale = 16
    )
    
    return ee.List([s2, s2_reduc])


# Apply the function to generate composites

# Get the fire IDs
fire_ids = fire_ids.slice(0,10)
print(fire_ids.getInfo())

# Run the function
results_list = fire_ids.map(lambda x: generate_composites(x))

# Extract sample and reference data for training and testing
images = results_list.map(lambda result: ee.Image(ee.List(result).get(0)))
reductions = results_list.map(lambda result: ee.FeatureCollection(ee.List(result).get(1)))

# Create the image collection
img_col = ee.ImageCollection.fromImages(images)
reduc = ee.FeatureCollection(reductions.flatten()) 

print(img_col.first().bandNames().getInfo())
print(reduc.first().getInfo())

print("Done!")

In [ ]:
# Initialize a map
Map = geemap.Map()
Map.addLayerControl()

f = img_col.first()
id = f.get('fired_id').getInfo()
print("Fire ID:", id)

# Grab the fire perimeter
fire = ee.FeatureCollection(fires.filter(ee.Filter.eq('fired_id', id)))
plots = frp.filter(ee.Filter.eq('fired_id', id))

Map.centerObject(plots)

# Visualize for one fire
vis_params_lai = {
    'bands': ['LAI'],
    'min': -50,
    'max': 100,
}
vis_params_mndwi = {
    'bands': ['MNDWI'],
    'min': -1,
    'max': 1,
}
vis_params_rgb = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Add the layers to the map
Map.addLayer(f, vis_params_lai, "LAI")
Map.addLayer(f, vis_params_mndwi, "MNDWI")
Map.addLayer(f, vis_params_rgb, "RGB")
Map.addLayer(fire.style(**{'color': 'orange', 'fillColor': '00000000'}), {}, 'Fire perimeter')
Map.addLayer(plots, {}, 'FRP Obs.')

Map

In [ ]:
# Check on the FRP reductions
print(reductions.first().getInfo())